In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spam-sms-classification-using-nlp/Spam_SMS.csv


In [2]:
data_df = pd.read_csv('/kaggle/input/spam-sms-classification-using-nlp/Spam_SMS.csv')
data_df.head()

,Class,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5574 entries, 0 to 5573
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Class    5574 non-null   object
 1   Message  5574 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [4]:
data_df.describe()

,Class,Message
count,5574,5574
unique,2,5159
top,ham,"Sorry, I'll call later"
freq,4827,30


In [5]:
data_df.isnull().sum()

Class      0
Message    0
dtype: int64

In [6]:
import string
from nltk.corpus import stopwords

In [7]:
def preprocess(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc=''.join(nopunc)
    return[word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [8]:
data_df['Message'].head().apply(preprocess)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: Message, dtype: object

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
messages=CountVectorizer(analyzer = preprocess)
messages.fit(data_df['Message'])
messages_vector=messages.transform(data_df['Message'])

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transform = TfidfTransformer().fit(messages_vector)

In [11]:
mess_tfidf_transform = tfidf_transform.transform(messages_vector)

In [12]:
print(mess_tfidf_transform)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 50547 stored elements and shape (5574, 11421)>
  Coords	Values
  (0, 11161)	0.23026810232333333
  (0, 10963)	0.19073747309736885
  (0, 8915)	0.24704694620162193
  (0, 8334)	0.17047287571100914
  (0, 7666)	0.2640334289254678
  (0, 7553)	0.3125357690523998
  (0, 6937)	0.18347278575774867
  (0, 6906)	0.15158985673221018
  (0, 6217)	0.1891588424976738
  (0, 5769)	0.24984740383972864
  (0, 5218)	0.26870529747253846
  (0, 5217)	0.29834974396666064
  (0, 4653)	0.3125357690523998
  (0, 2060)	0.2420402708624076
  (0, 1483)	0.3125357690523998
  (0, 1110)	0.28828459898893377
  (1, 11070)	0.4006170106480546
  (1, 10696)	0.20637541507921356
  (1, 8588)	0.5043365091357772
  (1, 7699)	0.37674276971445636
  (1, 3064)	0.29120672453799534
  (1, 2451)	0.5619816842520484
  (2, 11121)	0.1910440822579616
  (2, 11082)	0.1589832198667402
  (2, 10684)	0.13995809814630436
  :	:
  (5570, 6882)	0.31368079307544483
  (5570, 6691)	0.4778073440068897
  (5

In [13]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(mess_tfidf_transform,data_df['Class'])

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(mess_tfidf_transform,data_df['Class'],test_size=0.3)

In [15]:
model.fit(X_train,Y_train)

MultinomialNB()

In [16]:
predicted = model.predict(X_test)
original = Y_test

In [17]:
from sklearn.metrics import accuracy_score,classification_report
print("The accuracy of Model using Naive's Bayes model is : ",accuracy_score(predicted,original))
print(classification_report(predicted,original))

The accuracy of Model using Naive's Bayes model is :  0.949193066347878
              precision    recall  f1-score   support

         ham       1.00      0.94      0.97      1507
        spam       0.66      1.00      0.80       166

    accuracy                           0.95      1673
   macro avg       0.83      0.97      0.88      1673
weighted avg       0.97      0.95      0.95      1673



In [18]:
from sklearn.linear_model import LogisticRegression
logModel = LogisticRegression()
X_Train,X_Test,Y_Train,Y_Test = train_test_split(mess_tfidf_transform,data_df['Class'],test_size=0.3)

In [19]:
logModel.fit(X_Train,Y_Train)

LogisticRegression()

In [20]:
prediction = logModel.predict(X_Test)
expectation = Y_Test

In [21]:
print("Accuracy Scores with Logistics Regression :  ",accuracy_score(prediction,expectation))

Accuracy Scores with Logistics Regression :   0.931261207411835


In [22]:
print(classification_report(prediction,expectation))

              precision    recall  f1-score   support

         ham       1.00      0.93      0.96      1553
        spam       0.51      0.99      0.67       120

    accuracy                           0.93      1673
   macro avg       0.76      0.96      0.82      1673
weighted avg       0.96      0.93      0.94      1673

